# Introduction to GridAp 

In [1]:
using Gridap

## Section 1: Introduction 

### Questions
1. solve time-dependent problems using DifferentialEquations.jl; 
2. visualization using Makie 
3. define gauging field in 3D quasi-magnetic fields; 
4. define winding function (by solving electrostatic field) in 3D quasi-magnetic fields;

## Section 2: Scalar Diffusion Equation 

In [2]:
# Discrete model
n = 100
domain = (0,1,0,1)
partition = (n,n)
model = CartesianDiscreteModel(domain, partition)

CartesianDiscreteModel()

In [3]:
# write mesh to file 
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

In [4]:
# set exact solution and source term 
# f(x) is volumetric source term 
f(x) = 0 
# g(x) is the boundary conditions
g(x) = 1 
#g(x) = 0*(x[1]==0)+1*(x[1]==1)

# FEM approximation 
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags=[5])
# second position argument is function representing the Dirichlet boundary condition 
U = TrialFESpace(V0,g)

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

a(u,v) = ∫( ∇(v)⊙∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ

op = AffineFEOperator(a,b,U,V0)

uh = solve(op)

SingleFieldFEFunction():
 num_cells: 10000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 6846221987353836677

In [5]:
# write solution to file 
writevtk(Ω,"solution",cellfields=["uh" => uh])

(["solution.vtu"],)

In [7]:
?AffineFEOperator

search: AffineFEOperator TransientAffineFEOperator



```
AffineFEOperator
```

Represent a fully assembled affine (linear) finite element problem. See also [FEOperator](@ref)

---



In [12]:
edit(AffineFEOperator)

[1] AffineFEOperator(a::Function, ℓ::Function, args...) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:50
[2] AffineFEOperator(weakform::Function, trial::FESpace, test::FESpace, assem::Gridap.FESpaces.Assembler) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:23
[3] AffineFEOperator(weakform::Function, args...) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:44
[4] AffineFEOperator(trial::FESpace, test::FESpace, matrix::AbstractMatrix, vector::AbstractVector) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:16
[5] AffineFEOperator(trial::FESpace, test::FESpace, op::Gridap.Algebra.AffineOperator) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:9
[6] AffineFEOperator(trial, test, op) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/AffineFEOperators.jl:9

In [11]:
edit(FEOperator)

[1] FEOperator(res::Function, trial::FESpace, test::FESpace, assem::Gridap.FESpaces.Assembler) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/FEOperatorsFromWeakForm.jl:35
[2] FEOperator(res::Function, jac::Function, trial::FESpace, test::FESpace, assem::Gridap.FESpaces.Assembler) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/FEOperatorsFromWeakForm.jl:24
[3] FEOperator(res::Function, jac::Function, args...) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/FEOperatorsFromWeakForm.jl:29
[4] FEOperator(res::Function, args...) in Gridap.FESpaces at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/FESpaces/FEOperatorsFromWeakForm.jl:41

In [9]:
?FETerm

search: FETerm LinearFETerm AffineFETerm unsafe_pointer_to_objref



No documentation found.

`Gridap.FETerm` is a `Function`.

```
# 1 method for generic function "FETerm":
[1] FETerm(args...) in Gridap at /Users/djplahaye/.julia/packages/Gridap/9m0w9/src/Exports.jl:215
```


In [6]:
dump(op)

AffineFEOperator
  trial: TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}}
    dirichlet_values: Array{Float64}((99,)) [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
    space: Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}
      vector_type: Vector{Float64} <: DenseVector{Float64}
      nfree: Int64 10102
      ndirichlet: Int64 99
      cell_dofs_ids: Gridap.Arrays.Table{Int32, Vector{Int32}, Vector{Int32}}
        data: Array{Int32}((40000,)) Int32[1, -1, 3, 4, -1, -2, 4, 5, -2, -3  …  10099, 10100, 9999, 10000, 10100, 10101, 10000, 10001, 10101, 10102]
        ptrs: Array{Int32}((10001,)) Int32[1, 5, 9, 13, 17, 21, 25, 29, 33, 37  …  39965, 39969, 39973, 39977, 39981, 39985, 39989, 39993, 39997, 40001]
      fe_basis: Gridap.FESpaces.SingleFieldFEBasis{Gridap.FESpaces.TestBasis, ReferenceDomain}
        cell_basis: FillArrays

              tag_to_name: Array{String}((10,))
                1: String "tag_1"
                2: String "tag_2"
                3: String "tag_3"
                4: String "tag_4"
                5: String "tag_5"
                6: String "tag_6"
                7: String "tag_7"
                8: String "tag_8"
                9: String "interior"
                10: String "boundary"
          grid: CartesianGrid{2, Float64, typeof(identity)}
            node_coords: Gridap.Geometry.CartesianCoordinates{2, Float64, typeof(identity)}
              data: Gridap.Geometry.CartesianDescriptor{2, Float64, typeof(identity)}
                origin: VectorValue{2, Float64}
                sizes: Tuple{Float64, Float64}
                partition: Tuple{Int64, Int64}
                map: identity (function of type typeof(identity))
                isperiodic: Tuple{Bool, Bool}
            cell_node_ids: Gridap.Geometry.CartesianCellNodes{2}
              partition: Tuple{Int64, Int64}
   

              3: Array{Int32}((10000,)) Int32[9, 9, 9, 9, 9, 9, 9, 9, 9, 9  …  9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
            tag_to_entities: Array{Vector{Int32}}((10,))
              1: Array{Int32}((1,)) Int32[1]
              2: Array{Int32}((1,)) Int32[2]
              3: Array{Int32}((1,)) Int32[3]
              4: Array{Int32}((1,)) Int32[4]
              5: Array{Int32}((1,)) Int32[5]
              6: Array{Int32}((1,)) Int32[6]
              7: Array{Int32}((1,)) Int32[7]
              8: Array{Int32}((1,)) Int32[8]
              9: Array{Int32}((1,)) Int32[9]
              10: Array{Int32}((8,)) Int32[1, 2, 3, 4, 5, 6, 7, 8]
            tag_to_name: Array{String}((10,))
              1: String "tag_1"
              2: String "tag_2"
              3: String "tag_3"
              4: String "tag_4"
              5: String "tag_5"
              6: String "tag_6"
              7: String "tag_7"
              8: String "tag_8"
              9: String "interior"
              10: String 

          cell_type: FillArrays.Fill{Int8, 1, Tuple{Base.OneTo{Int64}}}
            value: Int8 1
            axes: Tuple{Base.OneTo{Int64}}
              1: Base.OneTo{Int64}
                stop: Int64 10000
        tface_to_mface: Gridap.Arrays.IdentityVector{Int64}
          length: Int64 10000
      domain_style: ReferenceDomain ReferenceDomain()
    cell_is_dirichlet: Array{Bool}((10000,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    dirichlet_dof_tag: Array{Int8}((99,)) Int8[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    dirichlet_cells: Array{Int32}((100,)) Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
    ntags: Int64 1
    metadata: Gridap.FESpaces.NodeToDofGlue{Int32}
      free_dof_to_node: Array{Int32}((10102,)) Int32[1, 101, 102, 103, 104, 105, 106, 107, 108, 109  …  10192, 10193, 10194, 10195, 10196, 10197, 10198, 10199, 10200, 10201]
      free_dof_to_comp: Array{Int16}((10102,)) Int16[1, 

In [10]:
?edit

search: edit @edit timedwait endswith redirect_stdio redirect_stdin



```
edit(path::AbstractString, line::Integer=0)
```

Edit a file or directory optionally providing a line number to edit the file at. Return to the `julia` prompt when you quit the editor. The editor can be changed by setting `JULIA_EDITOR`, `VISUAL` or `EDITOR` as an environment variable.

See also [`define_editor`](@ref).

---

```
edit(function, [types])
edit(module)
```

Edit the definition of a function, optionally specifying a tuple of types to indicate which method to edit. For modules, open the main source file. The module needs to be loaded with `using` or `import` first.

!!! compat "Julia 1.1"
    `edit` on modules requires at least Julia 1.1.


To ensure that the file can be opened at the given line, you may need to call `define_editor` first.
